# Distribution of influences

To get a grip on how much randomness influences the results, we run the same model lots of times and compare the results.

In [1]:
# for Colab, install fin_benefits and unemployment-gym from Github
#!pip install -q git+https://github.com/ajtanskanen/benefits.git  
#!pip install -q git+https://github.com/ajtanskanen/econogym.git
#!pip install -q git+https://github.com/ajtanskanen/lifecycle-rl.git

# and then restart kernel
  
  # For a specific version:
#!pip install tensorflow==1.15
#!pip install stable-baselines==2.8
  
# restart kernel after running pip's

Then load all modules and set parameters for simulations.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from lifecycle_rl import Lifecycle

%matplotlib inline
%pylab inline

# varoitukset piiloon (Stable baseline ei ole vielä Tensorflow 2.0-yhteensopiva, ja Tensorflow 1.15 valittaa paljon)
# ei taida toimia piilottaminen
import warnings
warnings.filterwarnings('ignore')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Populating the interactive namespace from numpy and matplotlib


In [3]:
# parameters for the simulation
# episode = 51 / 205 timesteps (1y/3m timestep)
pop_size=40_000 # size of the population to be simulated
size1=10_000_000 # number of timesteps in phase 1 training (callback not used)
size2=100 #0_000 # number of timesteps in phase 2 training (callback is used to save the best results)
size3=100 # number of timesteps in phase 1 training (callback not used) for policy changes
batch1=1_00 # size of minibatch in phase 1 as number of episodes
batch2=9_00  # size of minibatch in phase 1 as number of episodes
callback_minsteps=batch2 # how many episodes callback needs 
deterministic=False # use deterministic prediction (True) or probabilitic prediction (False)
mortality=False # include mortality in computations
randomness=True # include externally given, random state-transitions (parental leaves, disability, lay-offs) 
pinkslip=True # include lay-offs at 5 percent level each year
rlmodel='acktr' # use ACKTR algorithm
twostage=False # ajataan kahdessa vaiheessa vai ei
perusmalli='best/malli_baseline'
rlmodel='acktr' # use ACKTR algorithm
twostage=False # ajataan kahdessa vaiheessa vai ei
perusmalli_start='best/v2_malli_base_dev'
perusmalli='best/v2baseline/malli_base'
perusresults='results/v2baseline/distrib_baseline'
perusresults_stats=perusresults+'_stats'
debug=False # jos True, niin ajetaan vain yhdellä prosessilla. Nopeampi debugata.
plotdebug=False # tulostetaanko rivi riviltä mitä tapahtuu
learning_rate=0.06

# Baseline

Lasketaan työllisyysasteet nykymallissa.

In [4]:
n1=20
cc1=Lifecycle(env='unemployment-v2',minimal=False,mortality=mortality,perustulo=False,
              randomness=randomness,pinkslip=pinkslip,plotdebug=False)
cc1.explain()
cc1.run_distrib(n=n1,startn=14,debug=debug,steps1=size1,steps2=size2,pop=pop_size,deterministic=deterministic,
                train=True,predict=True,batch1=batch1,batch2=batch2,
                save=perusmalli,plot=True,cont=True,start_from=perusmalli_start,results=perusresults,
                callback_minsteps=callback_minsteps,rlmodel=rlmodel,twostage=twostage,learning_rate=learning_rate)


No mortality included
Parameters of lifecycle:
timestep 0.25
gamma 0.9793703613355593 (0.9200000000000003 per anno)
min_age 18
max_age 70
min_retirementage 63.5
max_retirementage 68
ansiopvraha_kesto300 300
ansiopvraha_kesto400 400
ansiopvraha_kesto500 500
ansiopvraha_toe 0.5
perustulo False
karenssi_kesto 0.25
mortality False
randomness True
include_putki True
include_pinkslip True
perustulo False

sigma_reduction True
plotdebug False

additional_tyel_premium 0.0
scale_tyel_accrual False

version 2
Parameters of lifecycle:
timestep 0.25
gamma 0.9793703613355593 (0.9200000000000003 per anno)
min_age 18
max_age 70
min_retirementage 63.5
max_retirementage 68
ansiopvraha_kesto300 300
ansiopvraha_kesto400 400
ansiopvraha_kesto500 500
ansiopvraha_toe 0.5
perustulo False
karenssi_kesto 0.25
mortality False
randomness True
include_putki True
include_pinkslip True
perustulo False

sigma_reduction True
plotdebug False

additional_tyel_premium 0.0
scale_tyel_accrual False

computing 14
train...


KeyboardInterrupt: 

In [ ]:
cc1.comp_distribs(load=perusresults,n=n1,startn=0,stats_results=perusresults+'_stats')

In [ ]:
cc2=Lifecycle(env='unemployment-v2',minimal=False,mortality=mortality,perustulo=False,
              randomness=randomness,pinkslip=pinkslip,plotdebug=False)
cc2.render_distrib(stats_results=perusresults_stats)

# Työssäoloehdon lyhentäminen

Asetetaan työssäoloehto 3 kuukauteen.

In [ ]:
notoemalli='best/notoe/malli_notoe'
notoeresults='results/notoe/distrib_notoe'
n=10
startn=0

In [ ]:
cc1_notoe=Lifecycle(env='unemployment-v2',minimal=False,mortality=mortality,
                    perustulo=False,randomness=randomness,ansiopvraha_toe=0.1)
cc1_notoe.run_distrib(n=n,startn=startn,debug=debug,steps1=size1,steps2=size2,pop=pop_size,deterministic=deterministic,
                train=True,predict=True,batch1=batch1,batch2=batch2,save=notoemalli,plot=True,cont=False,
                start_from=perusmalli_start,results=notoeresults,callback_minsteps=callback_minsteps,rlmodel=rlmodel,
                twostage=twostage,learning_rate=learning_rate)
cc1_notoe.comp_distribs(load=notoeresults,n=n,startn=0,stats_results=notoeresults+'_stats')
#cc1_notoe.render_distrib(notoeresults,n=n,startn=startn,stats_results=notoeresults+'_stats')

In [ ]:
cc1_notoe.comp_distribs(load=notoeresults,n=n,startn=0,stats_results=notoeresults+'_stats')

In [ ]:
cc1_notoe_b=Lifecycle(env='unemployment-v2',minimal=False,include_putki=False,mortality=mortality,
                    perustulo=False,randomness=randomness)
cc1_notoe_b.compare_distrib(perusresults_stats,notoeresults+'_stats')                    


# TOE 12 kk

Pidennetään työssäoloehto 12 kuukauteen

In [ ]:
toe12malli='best/toe12/malli_toe'
toe12results='results/toe12/malli_results'
n12=10

In [ ]:
cc2_toe=Lifecycle(env='unemployment-v2',minimal=False,ansiopvraha_toe=1.0,mortality=mortality,perustulo=False,randomness=randomness)
cc2_toe.run_distrib(n=n12,startn=0,debug=False,steps1=size1,steps2=size2,pop=pop_size,deterministic=deterministic,
                train=True,predict=True,batch1=10,batch2=batch2,
                save=toe12malli,plot=True,cont=True,start_from=perusmalli_start,results=toe12results,
                callback_minsteps=callback_minsteps,rlmodel=rlmodel,twostage=twostage,learning_rate=learning_rate)
cc2_toe.comp_distribs(load=toe12results,n=2,startn=0,stats_results=toe12results+'_stats')

In [ ]:
cc2_toe=Lifecycle(env='unemployment-v2',minimal=False,include_putki=False,mortality=mortality,perustulo=False,randomness=randomness)
cc2_toe.render_distrib(load=toe12results+'_v',n=n12,startn=0,stats_results=toe12results+'_stats')
cc2_toe.compare_distrib(perusresults_stats,toe12results+'_stats')